# Import wymaganych bibliotek

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from urllib.request import urlopen 
from pyspark.sql import SparkSession, Row
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from collections import OrderedDict
import json
import requests
import numpy
from datetime import datetime

In [3]:
spark = SparkSession.builder.appName("Analysis").getOrCreate()
sc = spark.sparkContext

In [4]:
sc.uiWebUrl

'http://10.0.2.15:4040'

# Deklaracja potrzebnych funkcji

In [5]:
def convert_to_row(d: dict) -> Row:
    return Row(**OrderedDict(d.items()))
#     return Row(**OrderedDict(sorted(d.items())))

Ogólna funkcja zwracająca RDD lub DF dla URL podanego jako argument

In [6]:
def get_data_from_ztm(url, resDf):
    response = urlopen(url)
    string_obj = response.read().decode('utf-8')
    dict_obj = json.loads(string_obj)
    
    l1 = dict_obj.get('result')
    l2 = list()
    l3 = list()
    d1 = dict()

    for v in l1:
        l2.append(v.get('values'))

    for v2 in l2:
        for elem in v2:
            d1[elem['key']] = elem['value']
        l3.append(d1.copy())
        
    rdd = spark.sparkContext.parallelize(l3)
    df = rdd.map(convert_to_row).toDF()
    
    if resDf:
        return df
    else:
        return rdd

Funkcja zwracająca RDD lub DF z przystankami

In [7]:
def get_bus_stops_ZTM(resDf):
    
    url = 'https://api.um.warszawa.pl/api/action/dbstore_get/?id=ab75c33d-3a26-4342-b36a-6e5fef0a3ac3&apikey=a9ee09f1-6975-46fc-ae46-df9b805ab6f6'
    response = urlopen(url)
    string_obj = response.read().decode('utf-8')
    dict_obj = json.loads(string_obj)
    
    l1= dict_obj.get('result')
    l2 = list()
    l3 = list()
    d1 = dict()

    for v in l1:
        l2.append(v.get('values'))

    for v2 in l2:
        for elem in v2:
            d1[elem['key']] = elem['value']
        l3.append(d1.copy())
        
    rdd = spark.sparkContext.parallelize(l3).map(convert_to_row)
    df = rdd.toDF()
    
    if resDf:
        return df
    else:
        return rdd
    

Pobranie linii autobusowych dostępnych na przystanku

In [8]:
def get_lines_on_bus_stop(busstopId, busstopNr):
    url = 'https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=88cd555f-6f31-43ca-9de4-66c479ad5942&busstopId=' + busstopId + '&busstopNr=' + busstopNr + '&apikey=a9ee09f1-6975-46fc-ae46-df9b805ab6f6'
    response = urlopen(url)
    string_obj = response.read().decode('utf-8')
    dict_obj = json.loads(string_obj)
    
    l1= dict_obj.get('result')
    l2 = list()
    l3 = list()
    d1 = dict()

    for v in l1:
        l2.append(v.get('values'))

    for v2 in l2:
        for elem in v2:
            d1[elem['key']] = elem['value']
        l3.append(d1.copy())
    
#     rdd = spark.sparkContext.parallelize(l3)#.map(convert_to_row)
    arr = numpy.array(l3)
    
    return arr

Pobranie rozkładu jazdy linii na przystanku

In [9]:
def get_schedule_of_line(busstopId, busstopNr, line):
    url = 'https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=' + busstopId + '&busstopNr=' + busstopNr + '&line=' + line + '&apikey=a9ee09f1-6975-46fc-ae46-df9b805ab6f6'
    response = urlopen(url)
    string_obj = response.read().decode('utf-8')
    dict_obj = json.loads(string_obj)
    
    l1= dict_obj.get('result')
    l2 = list()
    l3 = list()
    d1 = dict()

    for v in l1:
        l2.append(v.get('values'))

    for v2 in l2:
        for elem in v2:
            d1[elem['key']] = elem['value']
        l3.append(d1.copy())
    
#     rdd = spark.sparkContext.parallelize(l3)#.map(convert_to_row)
    arr = numpy.array(l3)
    
    return arr

Konwersja daty na wymagany format

In [10]:
test = '14:54:00'
def convDateTime(x):
    strDT =  f"{datetime.strftime(datetime.date(datetime.now()),'%Y-%m-%d')} {x}" 
    
    tmpMonth = int(strDT[5:7])
    tmpDay = int(strDT[8:10])
    tmpHour = int(strDT[11:13])
        
    if tmpHour != 24:
        return datetime.strptime(strDT,'%Y-%m-%d %H:%M:%S')
    else:
        newHour = '00'
        newDay = str(tmpDay + 1)
        if newDay != '32':
            resStr = strDT[:8] + newDay + strDT[10:11] + newHour + strDT[13:]
        else:
            newDay = '01'
            newMonth = str(tmpMonth + 1)
            resStr = strDT[:5] + newMonth + strDT[7:8] + newDay + strDT[10:11] + newHour + strDT[13:]
            
        return datetime.strptime(resStr,'%Y-%m-%d %H:%M:%S')
    
    
convDateTime(test)

datetime.datetime(2021, 5, 31, 14, 54)

Wyliczenie liczby sekund od 01.01.1970

In [11]:
def secsFrom1970(x):
    epoch = datetime.utcfromtimestamp(0)
    return (x - epoch).total_seconds()

# Pobranie współrzędnych przystanków 

In [12]:
# url = 'https://api.um.warszawa.pl/api/action/dbstore_get/?id=ab75c33d-3a26-4342-b36a-6e5fef0a3ac3&apikey=a9ee09f1-6975-46fc-ae46-df9b805ab6f6'
df_busStops = get_bus_stops_ZTM(True)
df_busStops = df_busStops.drop('id_ulicy', 'obowiazuje_od')

## Wyszukiwanie po nazwie przystanku

In [ ]:
df_bus.filter(df_bus.nazwa_zespolu=='Kazubów').show()

In [ ]:
df_bus.dtypes

# Pobranie danych do ewentualnej analizy (opcjonalne)

## Pobranie linii dostępnych na danym przystanku

In [ ]:
busstopId = '1001'
busstopNr = '01'
url = 'https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=88cd555f-6f31-43ca-9de4-66c479ad5942&busstopId=' + busstopId + '&busstopNr=' + busstopNr + '&apikey=a9ee09f1-6975-46fc-ae46-df9b805ab6f6'
df_lines = get_data_from_ztm(url, True)
df_lines.show()

## Pobranie rozkładu jazdy linii na przystanku

In [17]:
busstopId = '1001'
busstopNr = '01'
line = 'N71'
url = 'https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=' + busstopId + '&busstopNr=' + busstopNr + '&line=' + line + '&apikey=a9ee09f1-6975-46fc-ae46-df9b805ab6f6'
df_line = get_data_from_ztm(url, True)
df_line = df_line.drop('symbol_2', 'symbol_1')
df_line.show()

+-------+---------------+------+--------+
|brygada|       kierunek| trasa|    czas|
+-------+---------------+------+--------+
|    284|Wola Grzybowska|TP-ZOL|24:03:00|
|    185|Wola Grzybowska|TP-ZOL|25:03:00|
|    284|Wola Grzybowska|TP-ZOL|26:03:00|
|    185|Wola Grzybowska|TP-ZOL|27:03:00|
+-------+---------------+------+--------+



In [ ]:
df_line.count()

# Pobranie rozkładów jazdy linii na przystankach

## Pobranie i przygotowanie listy przystanków

In [13]:
url = 'https://api.um.warszawa.pl/api/action/dbstore_get/?id=ab75c33d-3a26-4342-b36a-6e5fef0a3ac3&apikey=a9ee09f1-6975-46fc-ae46-df9b805ab6f6'
df1 = get_data_from_ztm(url, True)

In [14]:
df1 = df1.select('zespol', 'slupek')

In [15]:
rdd1 = df1.rdd.map(list)
rdd1.take(5)

[['1001', '01'],
 ['1001', '02'],
 ['1001', '03'],
 ['1001', '04'],
 ['1001', '05']]

## Pobranie listy dostępnych linii dla każdego przystanku i dołączenie do DF 

In [16]:
rdd2 = rdd1.map(lambda x: [[x[0], x[1], line['linia']] for line in get_lines_on_bus_stop(x[0], x[1])])
rdd2.take(2)

[[['1001', '01', '123'],
  ['1001', '01', '125'],
  ['1001', '01', '138'],
  ['1001', '01', '146'],
  ['1001', '01', '147'],
  ['1001', '01', '166'],
  ['1001', '01', '202'],
  ['1001', '01', '509'],
  ['1001', '01', '902'],
  ['1001', '01', 'N02'],
  ['1001', '01', 'N03'],
  ['1001', '01', 'N21'],
  ['1001', '01', 'N71']],
 [['1001', '02', '138'],
  ['1001', '02', '166'],
  ['1001', '02', '509'],
  ['1001', '02', 'N21']]]

In [17]:
rdd2 = rdd2.flatMap(lambda x: x)
rdd2.take(2)

[['1001', '01', '123'], ['1001', '01', '125']]

In [18]:
df2 = rdd2.toDF(['zespol', 'slupek', 'linia'])
df2.printSchema()

root
 |-- zespol: string (nullable = true)
 |-- slupek: string (nullable = true)
 |-- linia: string (nullable = true)



In [19]:
df3 = df_busStops.join(df2, on = ['zespol', 'slupek'], how = 'outer')

In [20]:
df3.columns

['zespol',
 'slupek',
 'nazwa_zespolu',
 'szer_geo',
 'dlug_geo',
 'kierunek',
 'linia']

### Analiza (opcjonalne)

Rozkład partycji

In [ ]:
df3.rdd.glom().map(len).collect()

In [19]:
df3.show()

+------+------+-------------------+---------+---------+-----------------+-----+
|zespol|slupek|      nazwa_zespolu| szer_geo| dlug_geo|         kierunek|linia|
+------+------+-------------------+---------+---------+-----------------+-----+
|  1008|    03|              PIMot|52.270517|21.016711|       Golędzinów|   18|
|  1008|    03|              PIMot|52.270517|21.016711|       Golędzinów|   20|
|  1039|    03|      Księżnej Anny|52.261307|21.077520|       Nieświeska|  338|
|  1105|    01|           Ołówkowa|52.338677|20.992465|        Sadkowska|  152|
|  1105|    01|           Ołówkowa|52.338677|20.992465|        Sadkowska|  176|
|  1105|    01|           Ołówkowa|52.338677|20.992465|        Sadkowska|  N64|
|  1140|    01|       Metro Trocka|52.274977|21.056486|      Radzymińska|  140|
|  1140|    01|       Metro Trocka|52.274977|21.056486|      Radzymińska|  160|
|  1140|    01|       Metro Trocka|52.274977|21.056486|      Radzymińska|  199|
|  1140|    01|       Metro Trocka|52.27

In [21]:
df3.filter(df3.zespol.contains('1001') & df3.slupek.contains('01')).collect()

[Row(zespol='1001', slupek='01', nazwa_zespolu='Kijowska', szer_geo='52.248455', dlug_geo='21.044827', kierunek='al.Zieleniecka', linia='123'),
 Row(zespol='1001', slupek='01', nazwa_zespolu='Kijowska', szer_geo='52.248455', dlug_geo='21.044827', kierunek='al.Zieleniecka', linia='125'),
 Row(zespol='1001', slupek='01', nazwa_zespolu='Kijowska', szer_geo='52.248455', dlug_geo='21.044827', kierunek='al.Zieleniecka', linia='138'),
 Row(zespol='1001', slupek='01', nazwa_zespolu='Kijowska', szer_geo='52.248455', dlug_geo='21.044827', kierunek='al.Zieleniecka', linia='146'),
 Row(zespol='1001', slupek='01', nazwa_zespolu='Kijowska', szer_geo='52.248455', dlug_geo='21.044827', kierunek='al.Zieleniecka', linia='147'),
 Row(zespol='1001', slupek='01', nazwa_zespolu='Kijowska', szer_geo='52.248455', dlug_geo='21.044827', kierunek='al.Zieleniecka', linia='166'),
 Row(zespol='1001', slupek='01', nazwa_zespolu='Kijowska', szer_geo='52.248455', dlug_geo='21.044827', kierunek='al.Zieleniecka', linia=

In [23]:
df3.count()

22984

## Pobranie rozkładu jazdy dla linii na każdym przystanku i dołączenie do DF

In [21]:
rdd3 = rdd2.map(lambda x: [[x[0], x[1], x[2], elem['brygada'], elem['czas']] for elem in get_schedule_of_line(x[0], x[1], x[2])])

In [22]:
rdd3 = rdd3.flatMap(lambda x: x)

In [23]:
df4 = rdd3.toDF(['zespol', 'slupek', 'linia', 'brygada', 'czas'])

In [24]:
df3.printSchema()

root
 |-- zespol: string (nullable = true)
 |-- slupek: string (nullable = true)
 |-- nazwa_zespolu: string (nullable = true)
 |-- szer_geo: string (nullable = true)
 |-- dlug_geo: string (nullable = true)
 |-- kierunek: string (nullable = true)
 |-- linia: string (nullable = true)



In [25]:
df4.printSchema()

root
 |-- zespol: string (nullable = true)
 |-- slupek: string (nullable = true)
 |-- linia: string (nullable = true)
 |-- brygada: string (nullable = true)
 |-- czas: string (nullable = true)



In [26]:
df5 = df3.join(df4, on = ['zespol', 'slupek', 'linia'], how = 'outer')

In [27]:
df5.printSchema()

root
 |-- zespol: string (nullable = true)
 |-- slupek: string (nullable = true)
 |-- linia: string (nullable = true)
 |-- nazwa_zespolu: string (nullable = true)
 |-- szer_geo: string (nullable = true)
 |-- dlug_geo: string (nullable = true)
 |-- kierunek: string (nullable = true)
 |-- brygada: string (nullable = true)
 |-- czas: string (nullable = true)



## Odfiltrowanie linii z literami w nazwie oraz pustych rekordów

In [28]:
# Odfiltrowanie linii z literami w nazwie
df5 = df5.filter(df5.linia.rlike('^\d+'))
df5 = df5.filter(df5.szer_geo != 'null').filter(df5.dlug_geo != 'null')

## Obrobienie i podzielenie otrzymanego DF na bus i tram

In [29]:
rdd4 = df5.rdd.map(list)
rdd5 = rdd4.map(lambda x: [x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], f"{x[2]}/{x[7]}"])
rdd6 = rdd5.map(lambda x: [x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], f"{x[0]}/{x[1]}"])
rdd7 = rdd6.map(lambda x: [x[0], x[1], int(x[2]), x[3], float(x[4]), float(x[5]), x[6], x[7], x[8], x[9], x[10]])

In [30]:
df_all_1 = rdd7.toDF(['busStopGroupID', 'busStopNr', 'line', 'busStopName', 'sLat', 'sLon', 'dir', 'brigade', 'sTime', 'vehicleID', 'busStopID'])

KeyboardInterrupt: 

In [ ]:
df_all_2 = df_all_1.drop('busStopGroupID', 'busStopNr', 'dir',  'brigade')
df_all_2.count()

In [ ]:
df_all = df_all_2.dropna()
df_all.count()

In [ ]:
df_bus = df_all_tmp.filter(df_all_tmp.line >= 100)

In [ ]:
df_tram = df_all_tmp.filter(df_all_tmp.line < 100)

# Eksport otrzymanych DF do plików parquet

In [ ]:
df_bus.write.save('df_bus.parquet')

In [ ]:
df_tram.write.save('df_tram.parquet')

In [ ]:
df_all.write.save('df_all.parquet')